In [9]:
# import libraries
import pandas as pd
import numpy as np

In [10]:
# rates manually assigned to each report by Rachel
manual_polarity = pd.read_excel('Src/manual_polarity.xlsx')

### Open AI

In [17]:
# read openai polarity scores
openai_polarity = pd.read_csv('Scores/openai_polarity.csv')

# merge two dataframes on the basis of tweet_id
df = pd.merge(manual_polarity, openai_polarity, left_on='name', right_on="pdf_name", how='inner')
df.drop(['explanation'], axis=1, inplace=True)
df["polarity_y"] = df["polarity_y"].apply(lambda x: 0.5 if x == "Positive" else x)
df["polarity_y"] = pd.to_numeric(df["polarity_y"])
df.dropna(inplace=True)
df["polarity_y"].corr(df["rate"]) # find the correlation between OpenAI polarity and manual polarity

0.25137971975486223

### TextBlob

In [18]:
# polarity_textblob = pd.read_csv('Scores/textblob_polarity_2.csv')
polarity_textblob = pd.read_csv("Scores/polarity_textblob.csv")

df = pd.merge(manual_polarity, polarity_textblob, left_on='name', right_on="pdf_name", how='inner')
df.drop(['explanation'], axis=1, inplace=True)
df["polarity_y"] = df["polarity_y"].apply(lambda x: 0.5 if x == "Positive" else x)
df["polarity_y"] = pd.to_numeric(df["polarity_y"])
df.dropna(inplace=True)
df["polarity_y"].corr(df["rate"]) # find the correlation between TextBlob polarity and manual polarity

0.19554775742553324

In [19]:
df.head()

,name,rate,polarity_x,Unnamed: 0,pdf_name,polarity_y
0,COMBINED-Q4-17-Shareholder-Letter-FINAL,0.70,1,0,COMBINED-Q4-17-Shareholder-Letter-FINAL,0.142138
1,FINAL_Q2_15_Letter_to_Shareholders_With_Tables_,0.60,1,22,FINAL_Q2_15_Letter_to_Shareholders_With_Tables_,0.118486
2,FINAL_Q3_15_Letter_to_Shareholders_With_Tables_,0.55,0,23,FINAL_Q3_15_Letter_to_Shareholders_With_Tables_,0.098871
3,FINAL_Q3_Letter,0.60,1,24,FINAL_Q3_Letter,0.110256
4,FINAL-Q1-18-Shareholder-Letter,0.70,1,1,FINAL-Q1-18-Shareholder-Letter,0.106489


In [20]:
import statsmodels.api as sm

x = df[['polarity_y']]
y = df['rate']

# x = sm.add_constant(x)

model = sm.OLS(y, x).fit()
predictions = model.predict(x) 

print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:                   rate   R-squared (uncentered):                   0.957
Model:                            OLS   Adj. R-squared (uncentered):              0.956
Method:                 Least Squares   F-statistic:                              949.8
Date:                Fri, 18 Aug 2023   Prob (F-statistic):                    6.01e-31
Time:                        17:06:38   Log-Likelihood:                          25.969
No. Observations:                  44   AIC:                                     -49.94
Df Residuals:                      43   BIC:                                     -48.15
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

### Amazon

In [21]:
polarity_amazon = pd.read_csv('Scores/amazon_polarity.csv')

df = pd.merge(manual_polarity, polarity_amazon, left_on='name', right_on="pdf_name", how='inner')
df.drop(['explanation'], axis=1, inplace=True)
df["polarity_y"] = df["polarity_y"].apply(lambda x: 0.5 if x == "Positive" else x)
df["polarity_y"] = pd.to_numeric(df["polarity_y"])
df.dropna(inplace=True)
df["polarity_y"].corr(df["rate"])

-0.030976010219981146

In [62]:
df.head()

,name,rate,polarity_x,pdf_name,polarity_y
0,COMBINED-Q4-17-Shareholder-Letter-FINAL,0.70,1,COMBINED-Q4-17-Shareholder-Letter-FINAL,0.883941
1,FINAL_Q2_15_Letter_to_Shareholders_With_Tables_,0.60,1,FINAL_Q2_15_Letter_to_Shareholders_With_Tables_,0.897385
2,FINAL_Q3_15_Letter_to_Shareholders_With_Tables_,0.55,0,FINAL_Q3_15_Letter_to_Shareholders_With_Tables_,0.867095
3,FINAL_Q3_Letter,0.60,1,FINAL_Q3_Letter,0.881112
4,FINAL-Q1-18-Shareholder-Letter,0.70,1,FINAL-Q1-18-Shareholder-Letter,0.870285


In [22]:
import statsmodels.api as sm

x = df[['polarity_y']]
y = df['rate']

# x = sm.add_constant(x)

model = sm.OLS(y, x).fit()
predictions = model.predict(x) 

print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:                   rate   R-squared (uncentered):                   0.983
Model:                            OLS   Adj. R-squared (uncentered):              0.982
Method:                 Least Squares   F-statistic:                              2016.
Date:                Fri, 18 Aug 2023   Prob (F-statistic):                    1.56e-32
Time:                        17:06:46   Log-Likelihood:                          37.822
No. Observations:                  36   AIC:                                     -73.64
Df Residuals:                      35   BIC:                                     -72.06
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

### Google 

In [23]:
polarity_google = pd.read_csv('Scores/google_polarity.csv')

df = pd.merge(manual_polarity, polarity_google, left_on='name', right_on="pdf_name", how='inner')
df.drop(['explanation'], axis=1, inplace=True)
df["polarity_y"] = df["polarity_y"].apply(lambda x: 0.5 if x == "Positive" else x)
df["polarity_y"] = pd.to_numeric(df["polarity_y"])
df.dropna(inplace=True)
df["polarity_y"].corr(df["rate"])

0.4029289956880446

In [24]:
df.head()

,name,rate,polarity_x,pdf_name,polarity_y
0,COMBINED-Q4-17-Shareholder-Letter-FINAL,0.70,1,COMBINED-Q4-17-Shareholder-Letter-FINAL,0.2
1,FINAL_Q2_15_Letter_to_Shareholders_With_Tables_,0.60,1,FINAL_Q2_15_Letter_to_Shareholders_With_Tables_,0.2
2,FINAL_Q3_15_Letter_to_Shareholders_With_Tables_,0.55,0,FINAL_Q3_15_Letter_to_Shareholders_With_Tables_,0.1
3,FINAL_Q3_Letter,0.60,1,FINAL_Q3_Letter,0.2
4,FINAL-Q1-18-Shareholder-Letter,0.70,1,FINAL-Q1-18-Shareholder-Letter,0.2


In [25]:
import statsmodels.api as sm

x = df[['polarity_y']]
y = df['rate']

# x = sm.add_constant(x)

model = sm.OLS(y, x).fit()
predictions = model.predict(x) 

print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:                   rate   R-squared (uncentered):                   0.843
Model:                            OLS   Adj. R-squared (uncentered):              0.839
Method:                 Least Squares   F-statistic:                              230.8
Date:                Fri, 18 Aug 2023   Prob (F-statistic):                    6.83e-19
Time:                        17:06:52   Log-Likelihood:                         -2.3742
No. Observations:                  44   AIC:                                      6.748
Df Residuals:                      43   BIC:                                      8.533
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

### BART 

In [26]:
polarity_bart = pd.read_csv('Src/bart_positive_sentiment_probs.csv')

df = pd.merge(manual_polarity, polarity_bart, left_on='name', right_on="pdf_name", how='inner')
df.drop(['explanation'], axis=1, inplace=True)
df["positive_sentiment_prob"] = df["positive_sentiment_prob"].apply(lambda x: 0.5 if x == "Positive" else x)
df["positive_sentiment_prob"] = pd.to_numeric(df["positive_sentiment_prob"])
df.dropna(inplace=True)
df["positive_sentiment_prob"].corr(df["rate"])

0.22400821489739678

In [27]:
df.head()

,name,rate,polarity,pdf_name,positive_sentiment_prob
0,COMBINED-Q4-17-Shareholder-Letter-FINAL,0.70,1,COMBINED-Q4-17-Shareholder-Letter-FINAL,0.538636
1,FINAL_Q2_15_Letter_to_Shareholders_With_Tables_,0.60,1,FINAL_Q2_15_Letter_to_Shareholders_With_Tables_,0.539246
2,FINAL_Q3_15_Letter_to_Shareholders_With_Tables_,0.55,0,FINAL_Q3_15_Letter_to_Shareholders_With_Tables_,0.539036
3,FINAL_Q3_Letter,0.60,1,FINAL_Q3_Letter,0.539802
4,FINAL-Q1-18-Shareholder-Letter,0.70,1,FINAL-Q1-18-Shareholder-Letter,0.539028
